# Auth.js Authentication in Next.js (App Directory)

## Introduction
Auth.js (formerly NextAuth.js) simplifies authentication in Next.js applications by handling OAuth providers, sessions, and secure logins. This lecture demonstrates integrating **Auth.js** with the **Next.js App Directory** using **GitHub** as the OAuth provider. It covers project setup, configuration, and common pitfalls like server vs. client components.[1][2]

Key benefits include easy provider integration (GitHub, Google, etc.), automatic session management, and secure environment variables for client IDs/secrets. The tutorial uses Next.js 14+ features like `'use client'` directives.

## Prerequisites and Installation
- Create a new Next.js project: `npx create-next-app@latest next-auth-demo`.
- Install Auth.js: `npm install next-auth@beta` (beta for latest App Directory support).[3]
- Environment variables are stored in `.env.local` for local development and deployment platforms.

**Essential Environment Variables:**
- `AUTH_GITHUB_ID`: GitHub OAuth Client ID.
- `AUTH_GITHUB_SECRET`: GitHub OAuth Client Secret.
- `AUTH_SECRET`: Random secret for token encryption (generate with `npx auth secret`).[4]

Run `npm run dev` to start the development server on `http://localhost:3000`.

## Project Structure Setup
Create the Auth.js API route manually for App Directory compatibility (Pages Router docs may be outdated).

```
app/
├── api/
│   └── auth/
│       └── [...nextauth]/
│           └── route.ts  // Export handlers
├── layout.tsx            // Wrap app with SessionProvider
└── page.tsx              // Home page with sign-in/out
```

- **api/auth/[...nextauth]/route.ts**: 
  ```tsx
  import { handlers } from '@/auth'  // From root auth.ts
  export const { GET, POST } = handlers
  ```
  Export `GET` and `POST` handlers for Auth.js endpoints.[5][3]

## GitHub Provider Configuration
1. Log into GitHub → Settings → Developer Settings → OAuth Apps → New OAuth App.
2. **App Name**: e.g., "Sigma Web Dev".
3. **Homepage URL**: `http://localhost:3000`.
4. **Authorization Callback URL**: `http://localhost:3000/api/auth/callback/github`.
5. Copy **Client ID** and generate **Client Secret**.[6][1]

In root `auth.ts` (or `lib/auth.ts`):
```tsx
import NextAuth from 'next-auth'
import GitHub from 'next-auth/providers/github'

export const { handlers, auth, signIn, signOut } = NextAuth({
  providers: [GitHub]
})
```
Providers array accepts multiple (e.g., Google, Apple). Environment vars like `AUTH_GITHUB_ID` auto-map.[4]

## Session Management
Sessions use JWT by default (encrypted cookies). Wrap the app to provide session context.

### Client-Side SessionProvider
- Direct use in `layout.tsx` fails (server component error).
- Create `app/SessionWrapper.tsx` (client component):
  ```tsx
  'use client'
  import { SessionProvider } from 'next-auth/react'
  
  export function SessionWrapper({ children }: { children: React.ReactNode }) {
    return <SessionProvider>{children}</SessionProvider>
  }
  ```
- Update `layout.tsx`:
  ```tsx
  import { SessionWrapper } from './SessionWrapper'
  export default function RootLayout({ children }: { children: React.ReactNode }) {
    return (
      <html><body><SessionWrapper>{children}</SessionWrapper></body></html>
    )
  }
  ```
This enables `useSession()` hook app-wide.[7][1]

## Implementing Sign-In/Out UI
In `app/page.tsx` (mark as `'use client'`):
```tsx
'use client'
import { useSession, signIn, signOut } from 'next-auth/react'

export default function Home() {
  const { data: session } = useSession()
  if (session) {
    return (
      <>
        Signed in as {session.user?.email} [Image: {session.user?.image}]
        <button onClick={() => signOut()}>Sign out</button>
      </>
    )
  }
  return <button onClick={() => signIn('github')}>Sign in with GitHub</button>
}
```
- `useSession()` fetches session data (user email, image, etc.).
- `signIn('github')` redirects to GitHub OAuth.
- Direct provider: `signIn('github', { callbackUrl: '/' })` skips sign-in page.[1]

**Session Object**: Contains `user` (name, email, image) and `expires` timestamp. Log `console.log(session)` for full structure.[7]

## Common Errors and Fixes
| Error | Cause | Fix |
|-------|-------|-----|
| "React context unavailable in server components" | SessionProvider in server layout | Use client wrapper like SessionWrapper.tsx [1] |
| "Export named 'GET'/'POST'" | App Directory route handler | Export `handler as GET, handler as POST` [5] |
| No providers shown | Missing env vars | Set `AUTH_GITHUB_ID/SECRET` [4] |
| Callback mismatch | Wrong GitHub callback URL | Use `/api/auth/callback/github` [6] |

**Next.js Image**: Configure `next.config.js` for external images (e.g., GitHub avatars):
```js
images: { remotePatterns: [{ protocol: 'https', hostname: 'avatars.githubusercontent.com' }] }
```

## Summary
Auth.js enables quick OAuth authentication in Next.js App Directory via GitHub provider setup, env vars, and SessionProvider wrapping. Key steps: Install beta package, configure `auth.ts` with providers, create API route handlers, and use `useSession/signIn/signOut` in client components. Handles sessions securely with JWT; extend to multiple providers easily. Source code available in lecture GitHub repo for replication.